In [1]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup

# Specifying incognito mode as you launch your browser[OPTIONAL]
option = webdriver.ChromeOptions()
option.add_argument("--no-sandbox")

browser = webdriver.Chrome("/home/pahul/Documents/chromedriver")

companies_titles = ['groove',"slack"]
source_site_title = 'https://www.trustradius.com/'


data_final=[]
for company_title in companies_titles:
    browser.get('{}products/{}/reviews/alternatives-evaluated,likelihood-to-recommend,other-software-used,product-usage,pros-and-cons'.format(source_site_title, company_title))
    data_list= browser.find_elements_by_class_name("serp-details")
    for review in data_list:
        try:
            data_dict={"Company Name":'',"Review Title":'',"Reviewer's Name":'',"Reviewer's Position":'',"Reviewer's Company":'',"Score":'','Pros':'','Cons':'','Likelihood to Recommend':'','Use Cases and Deployment Scope':'','Alternatives Considered':'','Other Software Used':''}
            data_dict["Company Name"]=company_title
            data_dict["Review Title"]=review.find_element_by_class_name("review-title").text
            data_dict["Reviewer's Name"]=review.find_element_by_class_name("name").text
            data_dict["Reviewer's Position"]=review.find_element_by_class_name("position").text
            try:
                data_dict["Reviewer's Company"]=review.find_element_by_class_name("company").text
            except:
                data_dict["Reviewer's Company"]=review.find_element_by_class_name("industry-type").text
            data_dict["Score"]=review.find_element_by_class_name("trust-score__score").text
            questions=review.find_elements_by_class_name("question")
            responses=review.find_elements_by_class_name("response")
            for question,response in zip(questions,responses):
                if question.text=='Pros and Cons':
                    pros_list=review.find_element_by_class_name("pros").find_elements_by_class_name("ugc")
                    pros_data=[x.text for x in pros_list]
                    cons_list=review.find_element_by_class_name("cons").find_elements_by_class_name("ugc")
                    cons_data=[x.text for x in cons_list]
                    data_dict['Pros']=pros_data
                    data_dict['Cons']=cons_data

                if question.text=='Likelihood to Recommend':
                    data_dict["Likelihood to Recommend"]=response.text

                if question.text=="Use Cases and Deployment Scope":
                    data_dict["Use Cases and Deployment Scope"]=response.text

                if question.text=="Alternatives Considered":
                    data_dict["Alternatives Considered"]=response.text

                if question.text=="Other Software Used":
                    data_dict["Other Software Used"]=response.text

            data_final.append(data_dict)
        
        except:
            pass
    data_frame=pd.DataFrame(data_final)
    data_frame=data_frame[["Company Name","Reviewer's Name","Reviewer's Position","Reviewer's Company","Score","Review Title",'Pros','Cons','Likelihood to Recommend','Use Cases and Deployment Scope','Alternatives Considered','Other Software Used']]
    data_frame.to_csv("companies_trusted_scrapped_data.csv",index=False)
